<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/CCAP_UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import glob
import gdown
import zipfile
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import rasterio
import rasterio.plot
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import load_model
from keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
from scipy.ndimage import generic_filter
from scipy.stats import mode

# Additional code can be added here if needed

In [5]:
directory_path = '/content/drive/MyDrive/datahub_bucket/Extract_TrainingData'
os.makedirs(directory_path, exist_ok=True)

In [6]:
zip_files = glob.glob('/content/drive/MyDrive/datahub_bucket/TrainingData/*.zip')
extract_dir = '/content/drive/MyDrive/datahub_bucket/Extract_TrainingData' # destination directory
for zip_file in zip_files:
    base_name = os.path.basename(zip_file)[:-4]
    unzip_dir = os.path.join(extract_dir, base_name)
    os.makedirs(unzip_dir, exist_ok=True)
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(unzip_dir)

In [7]:
csv_files = glob.glob('/content/drive/MyDrive/datahub_bucket/TrainingData/*.csv')
csv_list = []
for csv_file in csv_files:
    base_name = os.path.basename(csv_file)[:-4]
    csv_dir = os.path.join(extract_dir, base_name)
    base_csv = pd.read_csv(csv_file)
    base_csv['subset'] = base_name
    csv_list.append(base_csv)
    index_csv = pd.concat(csv_list, ignore_index=True)

In [8]:
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).size().reset_index(name='Count')
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).agg({'percent': 'mean'}).reset_index()

In [9]:
pivot_df = unique_rows_df.pivot(index=['tif_name', 'subset'], columns='label', values='percent').reset_index().fillna(0)

In [10]:
merged_df = index_csv.merge(pivot_df, on=['tif_name', 'subset'], how='left')
merged_df.drop(columns=['label'], inplace=True)

In [11]:
sum_df = merged_df.drop_duplicates(subset=['tif_name', 'subset'])

In [12]:
sum_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,percent,tif_name,subset,13,14,15
0,5121,2049,4097,1025,37,749157.194721,751614.766259,2.322698e+06,2.325155e+06,0.002741,4097_1025.tif,Maui_oneYear_subset,0.002741,0.009592,0.006812
3,2049,3073,1025,2049,66,741784.411875,744242.028903,2.320240e+06,2.322698e+06,0.001051,1025_2049.tif,Maui_oneYear_subset,0.000000,0.000000,0.001051
4,4097,3073,3073,2049,68,746699.579012,749157.145333,2.320240e+06,2.322698e+06,0.096800,3073_2049.tif,Maui_oneYear_subset,0.096800,0.099092,0.007901
7,5121,3073,4097,2049,69,749157.176045,751614.789917,2.320240e+06,2.322698e+06,0.046784,4097_2049.tif,Maui_oneYear_subset,0.046784,0.035456,0.019820
10,6145,3073,5121,2049,70,751614.801963,754072.422910,2.320240e+06,2.322698e+06,0.082713,5121_2049.tif,Maui_oneYear_subset,0.082713,0.089973,0.017363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,28673,16385,27649,15361,508,805681.972014,808139.563877,2.288291e+06,2.290749e+06,0.034338,27649_15361.tif,Maui_oneYear_subset,0.034338,0.035327,0.000000
269,25601,17409,24577,16385,537,798309.191371,800766.782820,2.285834e+06,2.288291e+06,0.185758,24577_16385.tif,Maui_oneYear_subset,0.185758,0.000000,0.000000
270,26625,17409,25601,16385,538,800766.792034,803224.352420,2.285834e+06,2.288291e+06,0.462209,25601_16385.tif,Maui_oneYear_subset,0.462209,0.000000,0.000174
272,27649,17409,26625,16385,539,803224.350722,805681.983466,2.285834e+06,2.288291e+06,0.003212,26625_16385.tif,Maui_oneYear_subset,0.003212,0.000000,0.000000


In [13]:
total_rows = len(sum_df)
train_fraction = 0.9 # modify this to set the training percentage
train_rows = int(total_rows * train_fraction)
val_rows = total_rows - train_rows

random_assignment = np.array([0] * train_rows + [1] * val_rows)
np.random.seed(42)
np.random.shuffle(random_assignment)
sum_df['random_split'] = random_assignment

<ipython-input-13-66bd3969d95d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['random_split'] = random_assignment


In [21]:
sum_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,percent,tif_name,subset,13,14,15,random_split,Images_path,Labels_path
0,5121,2049,4097,1025,37,749157.194721,751614.766259,2.322698e+06,2.325155e+06,0.002741,4097_1025.tif,Maui_oneYear_subset,0.002741,0.009592,0.006812,0,/content/image_subsets/Maui_oneYear_subset/Ima...,/content/image_subsets/Maui_oneYear_subset/Lab...
3,2049,3073,1025,2049,66,741784.411875,744242.028903,2.320240e+06,2.322698e+06,0.001051,1025_2049.tif,Maui_oneYear_subset,0.000000,0.000000,0.001051,1,/content/image_subsets/Maui_oneYear_subset/Ima...,/content/image_subsets/Maui_oneYear_subset/Lab...
4,4097,3073,3073,2049,68,746699.579012,749157.145333,2.320240e+06,2.322698e+06,0.096800,3073_2049.tif,Maui_oneYear_subset,0.096800,0.099092,0.007901,0,/content/image_subsets/Maui_oneYear_subset/Ima...,/content/image_subsets/Maui_oneYear_subset/Lab...
7,5121,3073,4097,2049,69,749157.176045,751614.789917,2.320240e+06,2.322698e+06,0.046784,4097_2049.tif,Maui_oneYear_subset,0.046784,0.035456,0.019820,0,/content/image_subsets/Maui_oneYear_subset/Ima...,/content/image_subsets/Maui_oneYear_subset/Lab...
10,6145,3073,5121,2049,70,751614.801963,754072.422910,2.320240e+06,2.322698e+06,0.082713,5121_2049.tif,Maui_oneYear_subset,0.082713,0.089973,0.017363,0,/content/image_subsets/Maui_oneYear_subset/Ima...,/content/image_subsets/Maui_oneYear_subset/Lab...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,28673,16385,27649,15361,508,805681.972014,808139.563877,2.288291e+06,2.290749e+06,0.034338,27649_15361.tif,Maui_oneYear_subset,0.034338,0.035327,0.000000,0,/content/image_subsets/Maui_oneYear_subset/Ima...,/content/image_subsets/Maui_oneYear_subset/Lab...
269,25601,17409,24577,16385,537,798309.191371,800766.782820,2.285834e+06,2.288291e+06,0.185758,24577_16385.tif,Maui_oneYear_subset,0.185758,0.000000,0.000000,0,/content/image_subsets/Maui_oneYear_subset/Ima...,/content/image_subsets/Maui_oneYear_subset/Lab...
270,26625,17409,25601,16385,538,800766.792034,803224.352420,2.285834e+06,2.288291e+06,0.462209,25601_16385.tif,Maui_oneYear_subset,0.462209,0.000000,0.000174,0,/content/image_subsets/Maui_oneYear_subset/Ima...,/content/image_subsets/Maui_oneYear_subset/Lab...
272,27649,17409,26625,16385,539,803224.350722,805681.983466,2.285834e+06,2.288291e+06,0.003212,26625_16385.tif,Maui_oneYear_subset,0.003212,0.000000,0.000000,0,/content/image_subsets/Maui_oneYear_subset/Ima...,/content/image_subsets/Maui_oneYear_subset/Lab...


In [19]:
# agg_df = sum_df.groupby(['random_split']).agg({13: 'mean',14:'mean',15: 'mean',16:'mean',17: 'mean',18:'mean'}).reset_index()
agg_df = sum_df.groupby(['random_split']).agg({13: 'mean',14:'mean',15: 'mean'}).reset_index()
print(agg_df)

   random_split        13        14        15
0             0  0.228634  0.045680  0.003364
1             1  0.341195  0.124668  0.007790


In [63]:
sum_df['Images_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Image_Subset/"+sum_df['tif_name']
sum_df['Labels_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Label_Subset/"+sum_df['tif_name']

<ipython-input-63-acfe803626d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Images_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Image_Subset/"+sum_df['tif_name']
<ipython-input-63-acfe803626d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Labels_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Label_Subset/"+sum_df['tif_name']


In [64]:
sum_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,percent,tif_name,subset,13,14,15,random_split,Images_path,Labels_path
0,5121,2049,4097,1025,37,749157.194721,751614.766259,2.322698e+06,2.325155e+06,0.002741,4097_1025.tif,Maui_oneYear_subset,0.002741,0.009592,0.006812,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
3,2049,3073,1025,2049,66,741784.411875,744242.028903,2.320240e+06,2.322698e+06,0.001051,1025_2049.tif,Maui_oneYear_subset,0.000000,0.000000,0.001051,1,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
4,4097,3073,3073,2049,68,746699.579012,749157.145333,2.320240e+06,2.322698e+06,0.096800,3073_2049.tif,Maui_oneYear_subset,0.096800,0.099092,0.007901,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
7,5121,3073,4097,2049,69,749157.176045,751614.789917,2.320240e+06,2.322698e+06,0.046784,4097_2049.tif,Maui_oneYear_subset,0.046784,0.035456,0.019820,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
10,6145,3073,5121,2049,70,751614.801963,754072.422910,2.320240e+06,2.322698e+06,0.082713,5121_2049.tif,Maui_oneYear_subset,0.082713,0.089973,0.017363,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,28673,16385,27649,15361,508,805681.972014,808139.563877,2.288291e+06,2.290749e+06,0.034338,27649_15361.tif,Maui_oneYear_subset,0.034338,0.035327,0.000000,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
269,25601,17409,24577,16385,537,798309.191371,800766.782820,2.285834e+06,2.288291e+06,0.185758,24577_16385.tif,Maui_oneYear_subset,0.185758,0.000000,0.000000,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
270,26625,17409,25601,16385,538,800766.792034,803224.352420,2.285834e+06,2.288291e+06,0.462209,25601_16385.tif,Maui_oneYear_subset,0.462209,0.000000,0.000174,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
272,27649,17409,26625,16385,539,803224.350722,805681.983466,2.285834e+06,2.288291e+06,0.003212,26625_16385.tif,Maui_oneYear_subset,0.003212,0.000000,0.000000,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...


# the following begins the unet setup and execution

In [24]:
# Loading data from geotiff files
def load_data(files):
    data = []
    for file in files:
        with rasterio.open(file) as src:
            band_data = []
            for band in src.read():
                band_data.append(band)
            data.append(np.dstack(band_data))
    return np.array(data)

In [25]:
# example to show the width, height and bands of the images
def get_image_shapes_in_folders(folder_paths):
    image_shapes = []
    for folder_path in folder_paths:
        for root, dirs, files in os.walk(folder_path):
            # Sort the files alphabetically
            files = sorted(files)
            for file in files:
                if file.endswith('.tif') or file.endswith('.jpg') or file.endswith('.png'):
                    image_path = os.path.join(root, file)
                    try:
                        with rasterio.open(image_path) as src:
                            width, height = src.width, src.height
                            band_count = src.count  # Number of bands in the image
                            image_shapes.append((file, width, height, band_count))
                    except Exception as e:
                        print(f"Error getting shape of image '{file}': {e}")
    return image_shapes

folder_paths = ["/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Image_Subset"] # changed to Maui oneYear testing

shapes = get_image_shapes_in_folders(folder_paths)
for shape in shapes:
    file, width, height, band_count = shape
    print(f" {file[:-4]}: {width}, {height}, {band_count}")

 10241_10241: 1024, 1024, 7
 10241_11265: 1024, 1024, 7
 10241_12289: 1024, 1024, 7
 10241_5121: 1024, 1024, 7
 10241_6145: 1024, 1024, 7
 10241_9217: 1024, 1024, 7
 1025_2049: 1024, 1024, 7
 1025_4097: 1024, 1024, 7
 1025_6145: 1024, 1024, 7
 1025_7169: 1024, 1024, 7
 11265_10241: 1024, 1024, 7
 11265_6145: 1024, 1024, 7
 12289_10241: 1024, 1024, 7
 12289_5121: 1024, 1024, 7
 12289_6145: 1024, 1024, 7
 12289_7169: 1024, 1024, 7
 12289_8193: 1024, 1024, 7
 13313_5121: 1024, 1024, 7
 13313_6145: 1024, 1024, 7
 13313_7169: 1024, 1024, 7
 13313_8193: 1024, 1024, 7
 13313_9217: 1024, 1024, 7
 14337_5121: 1024, 1024, 7
 14337_6145: 1024, 1024, 7
 14337_7169: 1024, 1024, 7
 14337_8193: 1024, 1024, 7
 14337_9217: 1024, 1024, 7
 15361_4097: 1024, 1024, 7
 15361_5121: 1024, 1024, 7
 17409_4097: 1024, 1024, 7
 18433_10241: 1024, 1024, 7
 18433_5121: 1024, 1024, 7
 18433_6145: 1024, 1024, 7
 18433_7169: 1024, 1024, 7
 18433_8193: 1024, 1024, 7
 18433_9217: 1024, 1024, 7
 19457_10241: 1024, 1024, 

In [26]:
def load_and_reshape_image(image_path, img_height, img_width):
    with rasterio.open(image_path) as src:
        image = src.read()
        return image

In [29]:
load_and_reshape_image('/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Image_Subset/12289_6145.tif', 1024, 1024)

array([[[ 4.0771701e+02,  4.0534772e+02,  4.0298364e+02, ...,
          3.7885358e+02,  3.8050348e+02, -3.4000000e+38],
        [ 4.0486469e+02,  4.0265695e+02,  4.0045444e+02, ...,
          3.7974710e+02,  3.8147263e+02, -3.4000000e+38],
        [ 4.0200717e+02,  3.9996100e+02,  3.9792001e+02, ...,
          3.7878165e+02,  3.8036182e+02, -3.4000000e+38],
        ...,
        [ 3.9275848e+02,  3.9126236e+02,  3.8977042e+02, ...,
          3.4719907e+02,  3.4732098e+02,  3.4758734e+02],
        [ 3.9121930e+02,  3.8985376e+02,  3.8849243e+02, ...,
          3.4633377e+02,  3.4657089e+02,  3.4700220e+02],
        [ 3.8967593e+02,  3.8844098e+02,  3.8721027e+02, ...,
          3.4547006e+02,  3.4581543e+02,  3.4641171e+02]],

       [[ 7.1553882e+02,  7.1405023e+02,  7.1256525e+02, ...,
          6.3830017e+02,  6.3977313e+02, -3.4000000e+38],
        [ 7.1331860e+02,  7.1194342e+02,  7.1057190e+02, ...,
          6.3650983e+02,  6.3803430e+02, -3.4000000e+38],
        [ 7.1109479e+02, 

In [33]:
from tensorflow.keras.utils import to_categorical

In [34]:
def load_images_and_labels(image_files, label_files, img_height, img_width, num_classes):
    images = []
    labels = []

    for image_file in image_files:
        image = load_and_reshape_image(image_file, img_height, img_width)
        images.append(image)

    for label_file in label_files:
        label = load_and_reshape_image(label_file, img_height, img_width)
        # label -= 1  # adjust labels to be in the range 0-8 instead of 1-9 #

        # Add conditional statements to map labels to wetlands ('1') or non-wetlands ('0')
        # if label in [13, 14, 15, 16, 17, 18]:
        #     label = 1
        # else:
        #     label = 0
        # label = np.isin(label, [13, 14, 15, 16, 17, 18]).astype(int)
        label = np.isin(label, [13, 14, 15]).astype(int)
        label = to_categorical(label, num_classes=num_classes)  # one-hot encode the labels
        labels.append(label)

    return np.array(images), np.array(labels)

In [35]:
load_images_and_labels(['/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Image_Subset/12289_6145.tif'], ['/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Label_Subset/12289_6145.tif'], 1024, 1024, 7)

(array([[[[ 4.0771701e+02,  4.0534772e+02,  4.0298364e+02, ...,
            3.7885358e+02,  3.8050348e+02, -3.4000000e+38],
          [ 4.0486469e+02,  4.0265695e+02,  4.0045444e+02, ...,
            3.7974710e+02,  3.8147263e+02, -3.4000000e+38],
          [ 4.0200717e+02,  3.9996100e+02,  3.9792001e+02, ...,
            3.7878165e+02,  3.8036182e+02, -3.4000000e+38],
          ...,
          [ 3.9275848e+02,  3.9126236e+02,  3.8977042e+02, ...,
            3.4719907e+02,  3.4732098e+02,  3.4758734e+02],
          [ 3.9121930e+02,  3.8985376e+02,  3.8849243e+02, ...,
            3.4633377e+02,  3.4657089e+02,  3.4700220e+02],
          [ 3.8967593e+02,  3.8844098e+02,  3.8721027e+02, ...,
            3.4547006e+02,  3.4581543e+02,  3.4641171e+02]],
 
         [[ 7.1553882e+02,  7.1405023e+02,  7.1256525e+02, ...,
            6.3830017e+02,  6.3977313e+02, -3.4000000e+38],
          [ 7.1331860e+02,  7.1194342e+02,  7.1057190e+02, ...,
            6.3650983e+02,  6.3803430e+02, -3.4000

In [ ]:
# def load_images_and_labels(image_files, label_files, img_height, img_width, num_classes):
#   images = []
#   labels = []

#   # image_files = glob.glob(os.path.join(image_files, "*.tif"))
#   for image_file in image_files:
#       image = load_and_reshape_image(image_file, img_height, img_width)
#       images.append(image)

#   # label_files = glob.glob(os.path.join(label_files, "*.tif"))
#   for label_file in label_files:
#       label = load_and_reshape_image(label_file, img_height, img_width)
#       label -= 1  # adjust labels to be in the range 0-8 instead of 1-9
#       label = to_categorical(label, num_classes=num_classes)   # one-hot encode the labels
#       labels.append(label)

#   return np.array(images), np.array(labels)

In [86]:
class DataGenerator(Sequence):
    def __init__(self, image_files, label_files, img_height, img_width, batch_size, num_classes):
        self.image_files = image_files
        self.label_files = label_files
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        self.num_classes = num_classes
        # self.noise = noise

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.image_files[index * self.batch_size : (index + 1) * self.batch_size]
        batch_images, batch_labels = self.load_images_and_labels(batch_files)
        return batch_images, batch_labels

    def load_and_reshape_image(self, image_path):
        with rasterio.open(image_path) as src:
            image = src.read()
            image = image.transpose((1, 2, 0))
            if image.shape[0] != self.img_height or image.shape[1] != self.img_width:
                image = cv2.resize(image, (self.img_width, self.img_height), interpolation=cv2.INTER_NEAREST)
            if len(image.shape) == 3 and image.shape[2] == 1:
                image = np.squeeze(image, axis=2)

            return image

    def load_images_and_labels(self, image_files):
        images = []
        labels = []

        for image_file in image_files:
            image = self.load_and_reshape_image(image_file)
            image[image <= -3e+38] = np.nan

            # # Replace NaN values with the mean of the non-NaN pixels
            if np.any(np.isnan(image)):
                nan_mask = np.isnan(image)
                image[nan_mask] = np.nanmean(image)

            # # Replace Inf values with the mean of the non-Inf pixels
            if np.any(np.isinf(image)):
                inf_mask = np.isinf(image)
                image[inf_mask] = np.nanmean(image)

            # Convert to float
            image = image.astype(np.float32)

            # # Z-score normalization
            # mean = np.mean(image, axis=(0, 1), keepdims=True)
            # std = np.std(image, axis=(0, 1), keepdims=True)
            # # mean[mean < 0]
            # std[std < 0] = 0

            # # Normalize with epsilon to prevent divide by zero
            # epsilon = 1e-7

            # image = (image - mean) / (std + epsilon)

            images.append(image)

        for image_file in image_files:
            label_file = image_file.replace("Images", "Labels")
            label = self.load_and_reshape_image(label_file)
            # label = np.isin(label, [13, 14, 15, 16, 17, 18]).astype(int)
            label = np.isin(label, [13, 14, 15]).astype(int) # just for maui oneYear
            label = to_categorical(label, num_classes=4)  # one-hot encode the labels
            labels.append(label)

        return np.array(images), np.array(labels)

In [87]:
# import TenserFlow classes and functions
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization

In [88]:
def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model(num_classes):
	inputs = layers.Input(shape=[KERNEL_SIZE, KERNEL_SIZE, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(decoder0) # kept because this is a multiclass classification

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

In [89]:
def create_model(img_size, num_classes):
    inputs = keras.Input(shape=(img_size[0], img_size[1], 4))  # Change the number of channels to 4

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])  # updated to sparse categorical cross-entropy loss
    return model

In [41]:
# def predict_and_save_segments(input_folder, output_folder, model, img_height, img_width):
#       # Create output folder if it doesn't exist
#     os.makedirs(output_folder, exist_ok=True)

#     # Get a list of input files
#     input_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

#     for filename in input_files:
#               # Read input image
#         image_path = os.path.join(input_folder, filename)
#         with rasterio.open(image_path) as src:
#             # Read image data and reshape
#             image = src.read()  # Read all bands

#         image = load_and_reshape_image(image_path,img_height, img_width)
#         image = image.astype(np.uint8)
#         # print(image.shape)
#         masks = mask_generator.generate(image)

#         flat_mask = show_anns(masks)
#         imagery_file = rasterio.open(image_path)
#         imagery_transform = imagery_file.transform
#         reshaped_image = rasterio.plot.reshape_as_raster(flat_mask)
#         reshaped_image = reshaped_image[0]
#         # Get metadata from the input image
#         # print(reshaped_image.shape)
#         meta = src.meta

#         # Update metadata for the output image
#         meta.update(count=1, dtype=reshaped_image.dtype)

#         # Create output path
#         output_path = os.path.join(output_folder, filename)

#         # Write all 9 prediction channels as separate bands
#         with rasterio.open(output_path, 'w', **meta) as dst:
#             # for i in range(9):
#             dst.write(reshaped_image,1)  # Write each channel as a separate band

#         print(f"Saved prediction for {filename}")

#     print("Prediction and saving completed.")

In [77]:
def predict_and_save(input_folder, output_folder, model, img_height, img_width):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of input files
    input_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

    for filename in input_files:
        # Read input image
        input_path = os.path.join(input_folder, filename)
        with rasterio.open(input_path) as src:
            # Read image data and reshape
            image = src.read()  # Read all bands

            image[image <= -3e+38] = np.nan

            # # Replace NaN values with the mean of the non-NaN pixels
            if np.any(np.isnan(image)):
                nan_mask = np.isnan(image)
                image[nan_mask] = np.nanmean(image)

            # # Replace Inf values with the mean of the non-Inf pixels
            if np.any(np.isinf(image)):
                inf_mask = np.isinf(image)
                image[inf_mask] = np.nanmean(image)

            # Convert to float
            image = image.astype(np.float32)

            # # Z-score normalization
            # mean = np.mean(image, axis=(0, 1), keepdims=True)
            # std = np.std(image, axis=(0, 1), keepdims=True)
            # # mean[mean < 0]
            # std[std < 0] = 0

            # # Normalize with epsilon to prevent divide by zero
            # epsilon = 1e-7

            # image = (image - mean) / (std + epsilon)

            image = np.transpose(image, (1, 2, 0))  # Transpose to (height, width, bands)
            image = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
            image = np.expand_dims(image, axis=0)  # Add batch dimension
            # # Perform prediction
            prediction = model.predict(image)
            prediction[prediction <= 0] = np.nan
            prediction = prediction*255
            prediction = prediction.astype(np.uint8)

        # Get metadata from the input image
        meta = src.meta

        # Update metadata for the output image
        meta.update(count=26, dtype=prediction.dtype,nodata = 0)
        # meta.
        # Create output path
        output_path = os.path.join(output_folder, filename)

        # Write all 9 prediction channels as separate bands
        with rasterio.open(output_path, 'w', **meta) as dst:
            for i in range(2):
                dst.write(prediction[0, :, :, i], i + 1)  # Write each channel as a separate band

            # Add a 10th band containing the argmax of the 9 channels
            argmax_band = np.argmax(prediction[0], axis=-1)
            dst.write(argmax_band, 26)

        print(f"Saved prediction for {filename}")

    print("Prediction and saving completed.")

In [ ]:
# KERNEL_SIZE = 256 # was 512
# BANDS = range(7)
# KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
# OPTIMIZER = 'adam'
# LOSS = 'sparse_categorical_crossentropy'
# METRICS = ['categorical_accuracy']

In [90]:
KERNEL_SIZE = 1024 # was 512
BANDS = range(7)
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'
METRICS = ['categorical_accuracy']

In [91]:
model = get_model(num_classes =7)
print(model.summary())

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 1024, 1024, 7)]      0         []                            
                                                                                                  
 conv2d_138 (Conv2D)         (None, 1024, 1024, 32)       2048      ['input_7[0][0]']             
                                                                                                  
 batch_normalization_162 (B  (None, 1024, 1024, 32)       128       ['conv2d_138[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_162 (Activation  (None, 1024, 1024, 32)       0         ['batch_normalization_16

In [ ]:
# predict_and_save(input_folder, output_folder, model, img_height, img_width)
# predict_and_save('/content/image_subsets/Kauai_subset/Images/', '/content/Kauai_Predicts/', model, 512, 512)

In [92]:
# data_gen_test = DataGenerator(sum_df['Images_path'][0:5], sum_df['Labels_path'][0:5], 512, 512, 2, 22)
# Create the data generator
training_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==0], sum_df['Labels_path'][sum_df['random_split']==0], 1024, 1024, 7, 25) # was 512, 512, 4, 25
validation_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==1], sum_df['Labels_path'][sum_df['random_split']==1], 1024, 1024, 7, 25) # was 512, 512, 4, 25

In [93]:
# Get the number of batches in the training data generator
training_data_size = len(training_data_generator)

# Get the number of batches in the validation data generator
validation_data_size = len(validation_data_generator)

# Print the sizes
print("Training Data Generator Size:", training_data_size)
print("Validation Data Generator Size:", validation_data_size)


Training Data Generator Size: 19
Validation Data Generator Size: 3


In [94]:
batch_images, batch_labels = training_data_generator.__getitem__(0)

In [95]:
batch_images.shape

(7, 1024, 1024, 7)

In [96]:
batch_labels.shape

(7, 1024, 1024, 7, 4)

In [97]:
# Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) # removed 'accuracy', from metrics=[]
callbacks = [
    keras.callbacks.ModelCheckpoint("landcover_wetland", save_best_only=True)
]

model.fit(training_data_generator, validation_data=validation_data_generator, epochs=5, callbacks=callbacks,shuffle=True)
# model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=16, epochs=10)


Epoch 1/5


InvalidArgumentError: ignored

In [ ]:
predict_and_save('/content/image_subsets/Kauai_subset/Images/', '/content/Kauai_Predicts/', model, 256, 256) # was model, 512, 512)

In [ ]:
# Define the destination folder in Google Drive
import shutil
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/images'
source = '/content/Kauai_Predicts/5633_13825.tif'
# Define the destination path in Google Drive
destination_file = os.path.join(drive_folder, 'kauai_5633_13825.tif')  # Change 'your_file.tif' to your desired filename

# Copy the file to Google Drive
shutil.copyfile(source, destination_file)